In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard library imports.
import itertools

# Related third party imports.
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import gurobipy as gp
from gurobipy import GRB

# Local application/library specific imports.
from src.models.network_simulation import NetworkSimulation
import src.data.graph_utilities as gu

In [15]:
network_folder = '../data/processed/networks'
network_name = '26_usa'
factor = 0.5

# Load data
g = nx.read_gml(f'{network_folder}/{network_name}.gml', label="id")
diameter = gu.get_graph_diameter(g)
ml = diameter * factor

ns = NetworkSimulation(network_name, **{'max_length':ml, 'shortest_k':16})
S, H, C = list(g.nodes), list(g.nodes), list(g.nodes)
H_pairs = list(itertools.combinations(H, 2))
switch_H_pairs = gu.get_allowed_switch_hypervisor_pairs(ns.network_operator.triplets_by_switches)
# switch_H_pairs = {s:[(i,j) for _,i,j in ns.network_operator.triplets_by_switches[s] if (i!=s and j!=s)] for s in S}


In [16]:
model = gp.Model("mip1")

active_hypervisors = model.addVars(H, vtype=GRB.BINARY)
active_hypervisor_pairs = model.addVars(H_pairs, vtype=GRB.BINARY)

c_1 = model.addConstrs(active_hypervisor_pairs[(i,j)] == gp.min_(active_hypervisors[i], active_hypervisors[j]) for i,j in H_pairs)
c_2 = model.addConstrs(active_hypervisors[s] + gp.LinExpr([(1.0, active_hypervisor_pairs[(i,j)]) for i,j in switch_H_pairs[s]]) >= 1 for s in S)

model.setObjective(gp.quicksum(active_hypervisors), GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 26 rows, 351 columns and 2099 nonzeros
Model fingerprint: 0x6f02ece9
Model has 325 general constraints
Variable types: 0 continuous, 351 integer (351 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 26.0000000
Presolve added 418 rows and 0 columns
Presolve removed 0 rows and 116 columns
Presolve time: 0.01s
Presolved: 444 rows, 235 columns, 2935 nonzeros
Variable types: 0 continuous, 235 integer (235 binary)

Root relaxation: objective 1.166508e+00, 250 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.16651    0   92   26.00000    1.16651

In [17]:
for h,v in active_hypervisors.items():
    if v.x > 0.9:
        print('%s' % (h))
print('Obj: %g' % model.objVal)

1
2
4
6
Obj: 4
